In [51]:
import pandas as pd
from pathlib import Path
import json
import re

In [ ]:
with open('./data/course_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [53]:
df_courses = pd.DataFrame(data)
df_courses.head()

,search_term,url,university,faculty,field,Program Title,details
0,วิศวกรรมปัญญาประดิษฐ์,https://course.mytcas.com/programs/10100101300...,มหาวิทยาลัยสงขลานครินทร์,คณะวิศวกรรมศาสตร์,วิศวกรรมคอมพิวเตอร์,วศ.บ. วิศวกรรมปัญญาประดิษฐ์ (ภาษาไทย ปกติ) วิท...,{'ชื่อหลักสูตร': 'วศ.บ. วิศวกรรมปัญญาประดิษฐ์'...
1,วิศวกรรมปัญญาประดิษฐ์,https://course.mytcas.com/programs/10100329303...,มหาวิทยาลัยสงขลานครินทร์,วิทยาลัยการคอมพิวเตอร์,วิศวกรรมปัญญาประดิษฐ์,วศ.บ. วิศวกรรมปัญญาประดิษฐ์และระบบอัจฉริยะ (ภา...,{'ชื่อหลักสูตร': 'วศ.บ. วิศวกรรมปัญญาประดิษฐ์แ...
2,วิศวกรรมปัญญาประดิษฐ์,https://course.mytcas.com/programs/10140136303...,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,วิทยาลัยสหวิทยาการ,วิศวกรรมปัญญาประดิษฐ์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมระ...,{'ชื่อหลักสูตร': 'หลักสูตรวิศวกรรมศาสตรบัณฑิต ...
3,วิศวกรรมปัญญาประดิษฐ์,https://course.mytcas.com/programs/10160101303...,สถาบันเทคโนโลยีพระจอมเกล้าเจ้าคุณทหารลาดกระบัง,คณะวิศวกรรมศาสตร์,วิศวกรรมปัญญาประดิษฐ์,วศ.บ. วิศวกรรมปัญญาประดิษฐ์และการเป็นผู้ประกอบ...,{'ชื่อหลักสูตร': 'วศ.บ. วิศวกรรมปัญญาประดิษฐ์แ...
4,วิศวกรรมปัญญาประดิษฐ์,https://course.mytcas.com/programs/10180113303...,มหาวิทยาลัยอุบลราชธานี,คณะวิศวกรรมศาสตร์,วิศวกรรมปัญญาประดิษฐ์,วศ.บ.สาขาวิชาวิศวกรรมปัญญาประดิษฐ์และการสั่งกา...,{'ชื่อหลักสูตร': 'วศ.บ.สาขาวิชาวิศวกรรมปัญญาปร...


In [ ]:
# Filter out rows where 'faculty' does not contain "วิศวกรรม"
df_courses['faculty'] = df_courses['faculty'].astype(str)
df_courses = df_courses[df_courses['faculty'].str.contains('วิศวกรรม|สหวิทยาการ', na=False)]

In [55]:
fees_value = []
for entry in data:
    if 'details' in entry and 'ค่าใช้จ่าย' in entry['details']:
        fees_value.append(entry['details']['ค่าใช้จ่าย'])
    else:
        fees_value.append(None)

df_fees = pd.Series(fees_value, name='ค่าใช้จ่าย')

def classify_fees(value):
    if pd.isna(value):
        return "NaN"

    txt = str(value).strip() 
    link_pattern = re.compile(r"^(https?://|www\.)\S+")

    if link_pattern.match(txt):
        return "link"
    elif "ภาค" in txt or 'เทอม' in txt:
        return "ภาคการศึกษา"
    elif "ตลอดหลักสูตร" in txt:
        return "ตลอดหลักสูตร"
    elif txt.replace(",", "").isdigit() or 'บาท' in txt:
        return "คลุมเครือ"
    elif txt.lower() == "nan":
        return "NaN"
    else:
        return "อื่นๆ"
    
df_fees_classified = df_fees.apply(classify_fees)
print(df_fees_classified.value_counts())

df_courses['fees_type'] = df_fees.apply(classify_fees)

ค่าใช้จ่าย
ภาคการศึกษา     33
คลุมเครือ       12
ตลอดหลักสูตร     9
link             5
NaN              4
อื่นๆ            1
Name: count, dtype: int64


In [ ]:
# Find the link to fill out tuition information
df_links = df_courses[
    (df_courses['fees_type'] == 'link')  
].copy()
print(f"\n---{len(df_links)} rows---")
print(df_links[['details', 'fees_type']].to_string())


---3 rows---
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         details fees_type
2   {'ชื่อหลักสูตร': 'หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมระบบอัจฉริยะ ปริญญาตรี 4 ปี (พื้นที่การศึกษาราชบุรี)', 'ชื่อหลักสูตรภาษาอังกฤษ': 'Bachelor of Engineering Program in Intelligence Systems Engineering 4-year undergraduate program (Rajaburi educational region)', 'ประเภทหลักสูตร': 'ภาษาไทย ปกติ', 'สาขาวิชา': 'วิศวกรรมระบบอัจฉริยะ - วศ.บ 4 ปี (พื้นที่การศึกษาราชบุรี)', 'วิทยาเขต': 'บางมด', 'ค่าใช้จ่า

In [ ]:
# Fill out tuition information
link_tuitions_list = [25000, 70000, 30000]

df_links['ค่าใช้จ่ายต่อภาคการศึกษา'] = link_tuitions_list
display(df_links)

,search_term,url,university,faculty,field,Program Title,details,fees_type,ค่าใช้จ่ายต่อภาคการศึกษา
2,วิศวกรรมปัญญาประดิษฐ์,https://course.mytcas.com/programs/10140136303...,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,วิทยาลัยสหวิทยาการ,วิศวกรรมปัญญาประดิษฐ์,หลักสูตรวิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมระ...,{'ชื่อหลักสูตร': 'หลักสูตรวิศวกรรมศาสตรบัณฑิต ...,link,25000
23,วิศวกรรมคอมพิวเตอร์,https://course.mytcas.com/programs/10140107300...,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,คณะวิศวกรรมศาสตร์,วิศวกรรมคอมพิวเตอร์,วิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอมพิวเตอร...,{'ชื่อหลักสูตร': 'วิศวกรรมศาสตรบัณฑิต สาขาวิชา...,link,70000
24,วิศวกรรมคอมพิวเตอร์,https://course.mytcas.com/programs/10140107300...,มหาวิทยาลัยเทคโนโลยีพระจอมเกล้าธนบุรี,คณะวิศวกรรมศาสตร์,วิศวกรรมคอมพิวเตอร์,วิศวกรรมศาสตรบัณฑิต สาขาวิชาวิศวกรรมคอมพิวเตอร...,{'ชื่อหลักสูตร': 'วิศวกรรมศาสตรบัณฑิต สาขาวิชา...,link,30000


In [ ]:
def calculate_tuition(row):
    details = row.get('details')
    if not isinstance(details, dict):
        text = None
    else:
        text = details.get('ค่าใช้จ่าย')

    fees_type = row.get('fees_type')
    program_title = row.get('Program Title')

    if pd.isna(text) or text is None:
        return None

    text = str(text)

    match = re.search(r'(\d{1,3}(?:,\d{3})*(?:\.\d+)?)', text.replace('.-', '').replace(' บาท', '').replace('ประมาณ', '').strip())
    amount = None
    if match:
        try:
            amount = float(match.group(1).replace(',', ''))
        except ValueError:
            pass

    if amount is None or amount < 1000:
        return None

    tuition = None

    if fees_type == 'ภาคการศึกษา':
        tuition = amount
    elif fees_type == 'ตลอดหลักสูตร':
        if amount > 10000:
            tuition = amount // 8
        else:
            tuition = amount
    elif fees_type == 'คลุมเครือ':
        if program_title and 'ภาษาไทย ปกติ' in program_title and amount > 100000:
            tuition = amount // 8
        elif amount > 10000:
            tuition = amount
        else:
            tuition = amount

    return tuition

df_courses['ค่าใช้จ่ายต่อภาคการศึกษา'] = df_courses.apply(calculate_tuition, axis=1)

In [ ]:
df_courses = df_courses.dropna(subset=['ค่าใช้จ่ายต่อภาคการศึกษา'])

df_cleaned = pd.concat([df_courses, df_links], ignore_index=True)
df_cleaned.drop(columns=['fees_type'], inplace=True)

In [ ]:
output_json_path = './data/course_semester_tuition.json'
df_cleaned.to_json(output_json_path, orient='records', indent=4, force_ascii=False)
print(f"\nDataFrame with classified fees saved to {output_json_path}")


DataFrame with classified fees saved to ./data/course_semester_tuition.json
